# Desafío Número 1: Equipo 7

<a id="section_indice"></a> 
## Indice

[Introduccion](#section_introduccion)

[Dataset](#section_dataset)

[Limpieza de datos](#section_limpieza)

$\hspace{.5cm}$[Rooms](#section_limpieza_rooms)
  
$\hspace{.5cm}$[Expensas](#section_limpieza_expensas)

$\hspace{.5cm}$[Geonames](#section_limpieza_geonames)

$\hspace{.5cm}$[Pisos](#section_limpieza_pisos)

$\hspace{.5cm}$[Antigüedad](#section_limpieza_antiguedad)

$\hspace{.5cm}$[Precios](#section_limpieza_precios)

$\hspace{.5cm}$[Places](#section_limpieza_places)


[Definición de mitos o realidades](#section_mitos_o_realidades)

$\hspace{.5cm}$[Mito o Realidad N° 1: Los inmuebles a estrenar son más caros que los demás](#section_mito_1)

$\hspace{.5cm}$[Mito o Realidad N° 2: Existe una correlación entre el precio del inmueble y las expensas que abonan](#section_mito_2)



<a id="section_introduccion"></a> 

## Introducción

[volver a Indice](#section_indice)

El objetivo del desafío es generar la limpieza de una base de datos de "Properati" con observaciones sobre operaciones de venta de inmuebles aparentemente de Argentina.
Con el objetivo de visualizar los principales indicadores y responder una serie de preguntas planteadas, se procedió a la estrategia de limpieza.

Estrategia de limpieza: Se dividieron las columnas principales del dataset entre los integrantes del equipo, y se procedió a realizar la limpieza con prioridad en los datos existentes, complementando a través de regex los datos encontrados en las columnas "Title" y "Description".


<a id="section_dataset"></a> 

## Dataset

[volver a Indice](#section_indice)

In [ ]:
import pandas as pd
import numpy as np
import re 

In [ ]:
#Levantamos el archivo de trabajo
data_location = "../properati.csv"
data = pd.read_csv(data_location, sep=",")

#Analizamos el tipo de dato de cada variable
print(f'Tipo de datos de variables')
print(data.dtypes)


print('--------------------------------------------')
#Analizamos los nulos existentes en cada variable
print(f'Porcentaje de variables nulas')
nulos = data.isnull()
cant_no_nulos = round(nulos.sum()/data.shape[0]*100,2)
print(cant_no_nulos)

#Vusalizamos las primeras observaciones del data set
data.head(3)

<a id="section_limpieza"></a> 

## Limpieza del dataset de Properati

[volver a Indice](#section_indice)

<a id="section_limpieza_rooms"></a> 

### Limpieza de columna Rooms

[volver a Indice](#section_indice)

In [ ]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
data_room_null_title = data.title.astype(str).str.lower()
display(data_room_null_description)
display(data_room_null_title)

In [ ]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [ ]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [ ]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar)).astype(int)
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [ ]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'catorce':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [ ]:
#validar que luego elija siempre la última que encuentre

In [ ]:
def fun_buscar_disminuir_mapper_2(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
                'segundo': 2,'tercer':3, 'cuarto': 4,'quinto': 5,'sexto': 6
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [ ]:
def disminuciones_repeticion(serie,patt):
    print(data["rooms_completo"].isnull().value_counts())
    data_encontrada = serie.apply(lambda x: x if x is np.NaN else x.count(patt))
    maskarita= data_encontrada>0|data["rooms_completo"].isnull()
    data.loc[maskarita,"rooms_completo"] = data_encontrada[maskarita]
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [ ]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente|Monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

In [ ]:
data_room_match = fun_buscar_patt(data_room_null_title,r"(mono ambiente|monoambiente|mono-ambiente|monoamb)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d\d)[\s]?habitaci",0)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir_mapper_2(data_room_null_description,r"(\w+)[\s]?(dormitorio|domitorio)",0)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+)[\s]?(ambiente)",1)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_description,r"[\s](\d)[\s]?(amb)",1)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_description,r"\D(\d)[\s]?(dorm)",0)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_title,r"\D(\d?\d)[\s]?(dorm)",0)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_description,r"(\d)[\s]?(suite)",0)
print(resultado)

Buscamos ahora las restantes en el título

In [ ]:
resultado = fun_buscar_disminuir(data_room_null_title,r"([\s]\d?\d)[\s]?amb",1)
print(resultado)

In [ ]:
resultado = fun_buscar_disminuir_mapper(data_room_null_title,r"(\w+)[\s]?(amb)",1)
print(resultado)

In [ ]:
disminuciones_repeticion(data_room_null_description,"dormitorio")

In [ ]:
disminuciones_repeticion(data_room_null_description,"habitacion")


In [ ]:
disminuciones_repeticion(data_room_null_description,"suite")

## `SETEANDO RESTO DE ROOMS EN NULL`
ALGUNOS AJUSTES POSTERIORES ESPECÍFICOS PORQUE LA REGEX LOS TOMÓ Y NO APLICABA


AQUÍ SETEO EL RESTO QUE SE ENCUENTRA EN NULL

In [ ]:
def fun_back(idreg):
    data.loc[idreg:idreg,"rooms_completo"]=data.loc[idreg:idreg,"rooms"]
    return data.loc[idreg:idreg,"rooms_completo"]
fun_back(53793)
fun_back(23884)

In [ ]:
data["rooms_completo"].fillna(0,inplace = True) 
print(data["rooms_completo"].isnull().value_counts())

## `AGREGANDO CAMPOS NUEVOS`
Se crean nuevos campos para definir características que pueden revalorizar la propiedad
____

In [ ]:
data_encontrada1= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(pileta|piscina|pisina|picina)")
encontradas_pileta = data_encontrada1[data_encontrada1.notnull()].apply(lambda x: x.group(0))
data["PILETA"] = encontradas_pileta.apply(lambda x: False if x.count("sin")>0 else True)
data["PILETA"].fillna(value=False, inplace=True)
data["PILETA"].value_counts()

In [ ]:
data_encontrada_parrilla= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(parrilla|asado|grill|churrasquera)")
encontradas_parrilla = data_encontrada_parrilla[data_encontrada_parrilla.notnull()].apply(lambda x: x.group(0))
data["PARRILLA"] = encontradas_parrilla.apply(lambda x: False if x.count("sin")>0 else True)
data["PARRILLA"].fillna(value=False, inplace=True)
# data["PARRILLA"]= data_encontrada_parrilla.notnull()
data["PARRILLA"].value_counts()

In [ ]:
data_encontrada_patio= fun_buscar_patt(data_room_null_description,"(\w+)[\s]?(\w+)[\s]?(patio|jardin)")
encontradas_patio = data_encontrada_patio[data_encontrada_patio.notnull()].apply(lambda x: x.group(0))
data["PATIO_JARDIN"] = encontradas_patio.apply(lambda x: False if x.count("sin")>0 else True)
data["PATIO_JARDIN"].fillna(value=False, inplace=True)
data["PATIO_JARDIN"].value_counts()

ESTO ES PARA REALIZAR VALIDACIÓN DE DATOS


______


In [ ]:
bins= [1,2,3,4,5,6,7,8,9,10,15,20,25,30,40]
room= pd.cut(data.rooms_completo, bins,right=False)

In [ ]:
data.pivot_table('rooms_completo', index=[room], columns='property_type', aggfunc='count' )

<a id="section_limpieza_expensas"></a> 

### Limpieza de columna Expensas

[volver a Indice](#section_indice)


#### Vemos que porcentaje de las filas de expensas tienen NA y creamos un dataset solo con los que tienen datos

In [ ]:
#Separo los casos que son NA, los que tienen algun numero y de ellos los que son iguales a 1
mask_filas_con_expensas_na = data.expenses.isna()
mask_filas_con_expensas_notna = data.expenses.notna()
mask_filas_con_expensas_1 = data.expenses == 1

print(f'El total de filas es de {data.shape[0]}')
print(f'Hay {mask_filas_con_expensas_notna.sum()} filas con expensas, son el {round((mask_filas_con_expensas_notna.sum())/(data.shape[0]),2)*100}%')

#### Aplicamos la siguiente logica: si el tipo de propiedad es casa o PH, expensas = NO. Si es departamento o store = SI. Y si no tiene info tomamos el promedio de expensas segun los metros cuadrados.

In [ ]:
mask_apartment = data.property_type == "apartment"
mask_house = data.property_type == "house"
mask_PH = data.property_type == "PH"
mask_store = data.property_type == "store"

In [ ]:
data['expenses_bool'] = np.select([mask_apartment, mask_house, mask_PH, mask_store], 
                            ['SI', 'NO', 'NO', 'SI'], 
                            default=np.nan)
data.expenses_bool.value_counts()

#### De los casos que pagan expensas analizamos nuevamente los datos

In [ ]:
#Armo un df con los casos que pagan expensas
mask_si_expensas = data.expenses_bool == "SI"
con_expensas = data.loc[mask_si_expensas,:]
con_expensas['expenses'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
con_expensas.expenses.value_counts().sort_index()

#### Identificamos como desconocidos los casos atipicos (menor a 100 y mayor a 200.000) y los NA

In [ ]:
mask_mayor_doscientosmil = con_expensas.expenses <= 200000
mask_menor_cien = con_expensas.expenses >= 100

mask_conocidos = np.logical_and(mask_mayor_doscientosmil, mask_menor_cien)
mask_desconocidos = np.logical_not(mask_conocidos)

# con_expensas = con_expensas.loc[mask_quantil_noventaycinco, :]
# con_expensas.expenses.value_counts().sort_index()

In [ ]:
print(mask_conocidos.sum())
print(mask_desconocidos.sum())

In [ ]:
con_expensas_desconocidos = con_expensas.loc[mask_desconocidos,:]
con_expensas_conocidos = con_expensas.loc[mask_conocidos,:]
con_expensas_conocidos.expenses.describe().apply(lambda x: format(x, 'f'))

In [ ]:
con_expensas_conocidos.groupby('rooms')['expenses'].mean()

#### A los casos desconocidos les aplicamos el promedio por cantidad de habitaciones

In [ ]:
con_expensas_desconocidos = con_expensas_desconocidos.join(con_expensas_conocidos.groupby('rooms_completo')['expenses'].mean(), on='rooms_completo', rsuffix='_nuevo')

#### Unificamos los montos de expensas

In [ ]:
import warnings
warnings.filterwarnings("ignore")
con_expensas_conocidos.loc[:, 'expenses_nuevo'] = con_expensas_conocidos.loc[:, 'expenses'].copy()
con_expensas_conocidos.shape

In [ ]:
con_expensas_desconocidos.shape

In [ ]:
mask_no_expensas = data.expenses_bool == "NO"
sin_expensas = data.loc[mask_no_expensas,:]
sin_expensas.loc[:, 'expenses_nuevo'] = 0
sin_expensas.shape


In [ ]:
data = pd.concat([con_expensas_conocidos, con_expensas_desconocidos, sin_expensas])
data = data.sort_index()


<a id="section_limpieza_geonames"></a> 

### Limpieza de la columna Geonames

[volver a Indice](#section_indice)


In [ ]:
data_geolocalizacion = data.loc[ : , ["place_name", "place_with_parent_names", "geonames_id", "lat-lon", "lat", "lon"]]
display(data_geolocalizacion.head(10))
print(data_geolocalizacion.shape)

In [ ]:
nulos =data_geolocalizacion.isnull()
nulos.sum()

In [ ]:
geo_nulos = data_geolocalizacion.loc[nulos.index].sort_values('place_name')
geo_nulos.head()

In [ ]:
geo_nulos['place_name'].value_counts().sort_values(ascending=False).head(50)

### LIMPIEZA CORDOBA

In [ ]:
mask_cordoba = geo_nulos['place_name']== "Córdoba"
datos_cordoba = geo_nulos.loc[mask_cordoba, :]
#datos_cordoba.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_cordoba.isnull().sum()

In [ ]:
#display(datos_cordoba.groupby(['geonames_id']).mean())

In [ ]:
geo_mask_1 = datos_cordoba.place_with_parent_names == "|Argentina|Córdoba|Córdoba|"
geo_mask_2 = datos_cordoba.place_with_parent_names == "|Argentina|Córdoba|"

data_geolocalizacion_cordoba_3860259 = datos_cordoba.loc[geo_mask_1, :]
values_3860259 = {"geonames_id":3860259.0, "place_name":"Córdoba","place_with_parent_names": "|Argentina|Córdoba|Córdoba|", "lat-lon" : "-31.421912, -64.192741", "lat": -31.421912, "lon": -64.192741}

data_geolocalizacion_cordoba_3860255 = datos_cordoba.loc[geo_mask_2, :]
values_3860255 = {"geonames_id":3860255.0, "place_name":"Córdoba","place_with_parent_names": "|Argentina|Córdoba|","lat-lon" : "-31.411897, -64.269298", "lat": -31.411897, "lon": -64.269298}

datos_cordoba_con_3860259 = datos_cordoba.fillna(values_3860259)
datos_cordoba_completo = datos_cordoba.fillna(values_3860255)
#datos_cordoba_completo.head()


In [ ]:
#datos_cordoba_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_cordoba_completo)
geo_nulos.isnull().sum()

### LIMPIEZA ROSARIO

In [ ]:
mask_rosario = geo_nulos['place_name']== "Rosario"
datos_rosario = geo_nulos.loc[mask_rosario, :]
#datos_rosario.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_rosario.isnull().sum()

In [ ]:
#display(datos_rosario.groupby(['geonames_id']).mean())

In [ ]:
values_3838574 = {"geonames_id":3838574.0, "place_name":"Rosario","place_with_parent_names": "|Argentina|Santa Fe|Rosario|","lat-lon" : "-32.9473, -60.661795", "lat": -32.9473, "lon": -60.661795}
datos_rosario_completo = datos_rosario.fillna(values_3838574)
#datos_rosario_completo.head()

In [ ]:
#datos_rosario_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_rosario_completo)
geo_nulos.isnull().sum()

### LIMPIEZA MAR DEL PLATA

In [ ]:
mask_mar_del_plata = geo_nulos['place_name']== "Mar del Plata"
datos_mar_del_plata = geo_nulos.loc[mask_mar_del_plata, :]
#datos_mar_del_plata.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_mar_del_plata.isnull().sum()

In [ ]:
#display(datos_mar_del_plata.groupby(['geonames_id']).mean())

In [ ]:
values_3430863 = {"geonames_id":3430863.0, "place_name":"Mar del Plata","place_with_parent_names": "|Argentina|Buenos Aires Costa Atlántica|Mar de...","lat-lon" : "-38.006377, -57.553465", "lat": -38.006377, "lon": -57.553465}
datos_mar_del_plata_completo = datos_mar_del_plata.fillna(values_3430863)
#datos_mar_del_plata_completo.head()

In [ ]:
#datos_mar_del_plata_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_mar_del_plata_completo)
geo_nulos.isnull().sum()

### LIMPIEZA TIGRE

In [ ]:
mask_tigre = geo_nulos['place_name']== "Tigre"
datos_tigre = geo_nulos.loc[mask_tigre, :]
#datos_tigre.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_tigre.isnull().sum()

In [ ]:
#display(datos_tigre.groupby(['geonames_id']).mean())

In [ ]:
values_3427753 = {"geonames_id":3427753.0, "place_name":"Tigre","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|","lat-lon" : "-34.408622, -58.636493", "lat": -34.408622, "lon": -58.636493}
datos_tigre_completo = datos_tigre.fillna(values_3427753)
#datos_tigre_completo.head()

In [ ]:
#datos_tigre_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_tigre_completo)
geo_nulos.isnull().sum()

In [ ]:
mask_tigre2 = geo_nulos['place_with_parent_names'] == "|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||"
datos_tigre2 = geo_nulos.loc[mask_tigre2, :]
datos_tigre_completo2 = datos_tigre2.fillna(values_3427753)
datos_tigre_completo2.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_tigre_completo2)
geo_nulos.isnull().sum()

### LIMPIEZA NORDELTA

In [ ]:
mask_nordelta = geo_nulos['place_name']== "Nordelta"
datos_nordelta = geo_nulos.loc[mask_nordelta, :]
#datos_nordelta.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_nordelta.isnull().sum()

In [ ]:
#datos_nordelta.mean()

In [ ]:
values_3427753_Nord = {"geonames_id":3427753.0, "place_name":"Nordelta","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...","lat-lon" : "-34.410920, -58.644708", "lat": -34.410920, "lon": -58.644708}
datos_nordelta_completo = datos_nordelta.fillna(values_3427753_Nord)
#datos_nordelta_completo.head()

In [ ]:
#datos_nordelta_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_nordelta_completo)
geo_nulos.isnull().sum()

### LIMPIEZA BELGRANO

In [ ]:
mask_belgrano = geo_nulos['place_name']== "Belgrano"
datos_belgrano = geo_nulos.loc[mask_belgrano, :]
#datos_belgrano.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_belgrano.isnull().sum()

In [ ]:
#display(datos_belgrano.groupby(['geonames_id']).mean())

In [ ]:
values_3436077 = {"geonames_id":3436077.0, "place_name":"Belgrano","place_with_parent_names": "|Argentina|Capital Federal|Belgrano|","lat-lon" : "-34.563541, -58.453904", "lat": -34.563541, "lon": -58.453904}
datos_belgrano_completo = datos_belgrano.fillna(values_3436077)
#datos_belgrano_completo.head()

In [ ]:
#datos_belgrano_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_belgrano_completo)
geo_nulos.isnull().sum()

### LIMPIEZA PALERMO

In [ ]:
mask_palermo = geo_nulos['place_name']== "Palermo"
datos_palermo = geo_nulos.loc[mask_palermo, :]
#datos_palermo.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_palermo.isnull().sum()

In [ ]:
#display(datos_palermo.groupby(['geonames_id']).mean())

In [ ]:
values_3430234 = {"geonames_id":3430234.0, "place_name":"Palermo","place_with_parent_names": "|Argentina|Capital Federal|Palermo|","lat-lon" : "-34.583105, -58.424693", "lat": -34.583105, "lon": -58.424693}
datos_palermo_completo = datos_palermo.fillna(values_3430234)
#datos_palermo_completo.head()

In [ ]:
#datos_palermo_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_palermo_completo)
geo_nulos.isnull().sum()

### LIMPIEZA LA PLATA

In [ ]:
mask_la_plata = geo_nulos['place_name']== "La Plata"
datos_la_plata = geo_nulos.loc[mask_la_plata, :]
#datos_la_plata.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_la_plata.isnull().sum()

In [ ]:
#display(datos_la_plata.groupby(['geonames_id']).mean())

In [ ]:
values_3432039 = {"geonames_id":3432039.0, "place_name":"La Plata","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|","lat-lon" : "-34.924338, -57.966073", "lat": -34.924338, "lon": -57.966073}
datos_la_plata_completo = datos_la_plata.fillna(values_3432039)
#datos_la_plata_completo.head()

In [ ]:
#datos_la_plata_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_la_plata_completo)
geo_nulos.isnull().sum()

### LIMPIEZA CABALLITO

In [ ]:
mask_caballito = geo_nulos['place_name']== "Caballito"
datos_caballito = geo_nulos.loc[mask_caballito, :]
#datos_caballito.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_caballito.isnull().sum()

In [ ]:
#display(datos_caballito.groupby(['geonames_id']).mean())

In [ ]:
values_3435874 = {"geonames_id":3435874.0, "place_name":"Caballito","place_with_parent_names": "|Argentina|Capital Federal|Caballito|","lat-lon" : "-34.618748, -58.442827", "lat": -34.618748, "lon": -58.442827}
datos_caballito_completo = datos_caballito.fillna(values_3435874)
#datos_caballito_completo.head()

In [ ]:
#datos_caballito_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_caballito_completo)
geo_nulos.isnull().sum()

### LIMPIEZA PILAR

In [ ]:
mask_pilar = geo_nulos['place_name']== "Pilar"
datos_pilar = geo_nulos.loc[mask_pilar, :]
#datos_pilar.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_pilar.isnull().sum()

In [ ]:
#display(datos_pilar.groupby(['geonames_id']).mean())

In [ ]:
#display(datos_pilar.groupby(['geonames_id']).count())

In [ ]:
values_3429979 = {"geonames_id":3429979.0, "place_name":"Pilar","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|","lat-lon" : "-34.448529, -58.868267", "lat": -34.448529, "lon": -58.868267}
datos_pilar_completo = datos_pilar.fillna(values_3429979)
#datos_pilar_completo.head()

In [ ]:
#datos_pilar_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_pilar_completo)
geo_nulos.isnull().sum()

### LIMPIEZA VILLA URQUIZA

In [ ]:
mask_villa_urquiza = geo_nulos['place_name']== "Villa Urquiza"
datos_villa_urquiza = geo_nulos.loc[mask_villa_urquiza, :]
#datos_villa_urquiza.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_villa_urquiza.isnull().sum()

<!--     original	bajado	saldo
geo: 18717 - 4648 - 14069/
lat-lon: 51550 - 19491 - 32059/
lat: 51550 - 19491 - 32059/
lon: 51550 - 19491 - 32059 -->

In [ ]:
#display(datos_villa_urquiza.groupby(['geonames_id']).mean())

In [ ]:
values_3433775 = {"geonames_id":3433775.0, "place_name":"Villa Urquiza","place_with_parent_names": "|Argentina|Capital Federal|Villa Urquiza|","lat-lon" : "-34.573057, -58.483033", "lat": -34.573057, "lon": -58.483033}
datos_villa_urquiza_completo = datos_villa_urquiza.fillna(values_3429979)
#datos_villa_urquiza_completo.head()

In [ ]:
#datos_villa_urquiza_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_villa_urquiza_completo)
geo_nulos.isnull().sum()

### LIMPIEZA OLIVOS

In [ ]:
mask_olivos = geo_nulos['place_name']== "Olivos"
datos_olivos = geo_nulos.loc[mask_olivos, :]
#datos_olivos.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_olivos.isnull().sum()

In [ ]:
#display(datos_olivos.groupby(['geonames_id']).mean())

In [ ]:
values_3430310 = {"geonames_id":3430310.0, "place_name":"Olivos","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...","lat-lon" : "-34.510486, -58.490124", "lat": -34.510486, "lon": -58.490124}
datos_olivos_completo = datos_olivos.fillna(values_3429979)
#datos_olivos_completo.head()

In [ ]:
#datos_olivos_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_olivos_completo)
geo_nulos.isnull().sum()

### LIMPIEZA RECOLETA

In [ ]:
mask_recoleta = geo_nulos['place_name']== "Recoleta"
datos_recoleta = geo_nulos.loc[mask_recoleta, :]
#datos_recoleta.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_recoleta.isnull().sum()

In [ ]:
#display(datos_recoleta.groupby(['geonames_id']).mean())

In [ ]:
values_3429595 = {"geonames_id":3429595.0, "place_name":"Recoleta","place_with_parent_names": "|Argentina|Capital Federal|Recoleta|","lat-lon" : "-34.592, -58.395299", "lat": -34.592, "lon": -58.395299}
datos_recoleta_completo = datos_recoleta.fillna(values_3429595)
#datos_recoleta_completo.head()

In [ ]:
#datos_recoleta_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_recoleta_completo)
geo_nulos.isnull().sum()

### LIMPIEZA LOMAS DE ZAMORA

In [ ]:
mask_lomas_de_zamora = geo_nulos['place_name']== "Lomas de Zamora"
datos_lomas_de_zamora = geo_nulos.loc[mask_lomas_de_zamora, :]
#datos_lomas_de_zamora.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_lomas_de_zamora.isnull().sum()

In [ ]:
#display(datos_lomas_de_zamora.groupby(['geonames_id']).mean())

In [ ]:
values_3431270 = {"geonames_id":3431270.0, "place_name":"Lomas de Zamora","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...","lat-lon" : "-34.76043, -58.412512", "lat": -34.76043, "lon": -58.412512}
datos_lomas_de_zamora_completo = datos_lomas_de_zamora.fillna(values_3431270)
#datos_lomas_de_zamora_completo.head()

In [ ]:
#datos_lomas_de_zamora_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_lomas_de_zamora_completo)
geo_nulos.isnull().sum()

### LIMPIEZA FLORES

In [ ]:
mask_flores = geo_nulos['place_name']== "Flores"
datos_flores = geo_nulos.loc[mask_flores, :]
#datos_flores.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_flores.isnull().sum()

In [ ]:
#display(datos_flores.groupby(['geonames_id']).mean())

In [ ]:
values_3433918 = {"geonames_id":3433918.0, "place_name":"Flores","place_with_parent_names": "|Argentina|Capital Federal|Flores|","lat-lon" : "-34.628981, -58.46387", "lat": -34.628981, "lon": -58.46387}
datos_flores_completo = datos_flores.fillna(values_3431270)
#datos_flores_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_flores_completo)
geo_nulos.isnull().sum()

### LIMPIEZA ITUZAINGO

In [ ]:
mask_ituzaingo = geo_nulos['place_name']== "Ituzaingó"
datos_ituzaingo = geo_nulos.loc[mask_ituzaingo, :]
#datos_ituzaingo.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_ituzaingo.isnull().sum()

In [ ]:
#display(datos_ituzaingo.groupby(['geonames_id']).mean())

In [ ]:
geo_mask_3 = datos_ituzaingo.place_with_parent_names == "|Argentina|Corrientes|Ituzaingó|"
geo_mask_4 = datos_ituzaingo.place_with_parent_names == "|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|"

datos_ituzaingo_3433357 = datos_ituzaingo.loc[geo_mask_3, :]
values_3433357 = {"geonames_id":3433357.0, "place_name":"Ituzaingó","place_with_parent_names": "|Argentina|Corrientes|Ituzaingó|", "lat-lon" : "-27.573083, -56.705464", "lat": -27.573083, "lon": -56.705464}


datos_ituzaingo_3433359 = datos_ituzaingo.loc[geo_mask_4, :]
values_3860259 = {"geonames_id":3860259.0, "place_name":"Ituzaingó","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó||","lat-lon" : "-34.652894, -58.673752", "lat": -34.652894, "lon": -58.673752}


datos_ituzaingo_33433357 = datos_ituzaingo.fillna(values_3433357)
datos_ituzaingo_completo = datos_ituzaingo.fillna(values_3860259)
#datos_ituzaingo_completo.head()

In [ ]:
#datos_ituzaingo_completo.isnull().sum()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_ituzaingo_completo)
geo_nulos.isnull().sum()

### LIMPIEZA VILLA CRESPO

In [ ]:
mask_villa_crespo = geo_nulos['place_name']== "Villa Crespo"
datos_villa_crespo = geo_nulos.loc[mask_villa_crespo, :]
#datos_villa_crespo.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_villa_crespo.isnull().sum()

In [ ]:
#display(datos_villa_crespo.groupby(['geonames_id']).mean())

In [ ]:
values_3427458 = {"geonames_id":3427458.0, "place_name":"Villa Crespo","place_with_parent_names": "|Argentina|Capital Federal|Villa Crespo|","lat-lon" : "-34.597889, -58.439293", "lat": -34.597889, "lon": -58.439293}
datos_villa_crespo_completo = datos_villa_crespo.fillna(values_3427458)
#datos_villa_crespo_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_villa_crespo_completo)
geo_nulos.isnull().sum()

### LIMPIEZA CAPITAL FEDEREAL

In [ ]:
mask_capital_federal = geo_nulos['place_name']== "Capital Federal"
datos_capital_federal = geo_nulos.loc[mask_capital_federal, :]
#datos_capital_federal.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_capital_federal.isnull().sum()

In [ ]:
#display(datos_capital_federal.groupby(['geonames_id']).mean())

In [ ]:
values_3433955 = {"geonames_id":3433955.0, "place_name":"Capital Federal","place_with_parent_names": "|Argentina|Capital Federal|","lat-lon" : "-34.626709, -58.435882", "lat": -34.626709, "lon": -58.435882}
datos_capital_federal_completo = datos_capital_federal.fillna(values_3433955)
#datos_capital_federal_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_capital_federal_completo)
geo_nulos.isnull().sum()

### LIMPIEZA SAN TELMO

In [ ]:
mask_san_telmo = geo_nulos['place_name']== "San Telmo"
datos_san_telmo = geo_nulos.loc[mask_san_telmo, :]
#datos_san_telmo.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_san_telmo.isnull().sum()

In [ ]:
#display(datos_san_telmo.groupby(['geonames_id']).mean())

In [ ]:
values_3428113 = {"geonames_id":3428113.0, "place_name":"San Telmo","place_with_parent_names": "|Argentina|Capital Federal|San Telmo|","lat-lon" : "-34.620805, -58.376068", "lat": -34.620805, "lon": 	-58.376068}
datos_san_telmo_completo = datos_san_telmo.fillna(values_3428113)
#datos_san_telmo_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_san_telmo_completo)
geo_nulos.isnull().sum()

### LIMPIEZA MORON

In [ ]:
mask_moron = geo_nulos['place_name']== "Morón"
datos_moron = geo_nulos.loc[mask_moron, :]
#datos_moron.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_moron.isnull().sum()

In [ ]:
#display(datos_moron.groupby(['geonames_id']).mean())

In [ ]:
values_3430544 = {"geonames_id":3430544.0, "place_name":"Morón","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|","lat-lon" : "-34.654915, -58.62302", "lat": -34.654915, "lon": -58.62302}
datos_moron_completo = datos_moron.fillna(values_3430544)
#datos_moron_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_moron_completo)
geo_nulos.isnull().sum()

### LIMPIEZA ALMAGRO

In [ ]:
mask_almagro = geo_nulos['place_name']== "Almagro"
datos_almagro = geo_nulos.loc[mask_almagro, :]
#datos_almagro.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_almagro.isnull().sum()

In [ ]:
#display(datos_almagro.groupby(['geonames_id']).mean())

In [ ]:
values_3436397 = {"geonames_id":3436397.0, "place_name":"Almagro","place_with_parent_names": "|Argentina|Capital Federal|Almagro|","lat-lon" : "-34.608187, -58.421223", "lat": -34.608187, "lon": -58.421223}
datos_almagro_completo = datos_almagro.fillna(values_3436397)
#datos_almagro_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_almagro_completo)
geo_nulos.isnull().sum()

### LIMPIEZA BARRIO NORTE

In [ ]:
mask_barrio_norte = geo_nulos['place_name']== "Barrio Norte"
datos_barrio_norte = geo_nulos.loc[mask_barrio_norte, :]
#datos_barrio_norte.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_barrio_norte.isnull().sum()

In [ ]:
#display(datos_barrio_norte.groupby(['geonames_id']).mean())

In [ ]:
values_3436109 = {"geonames_id":3436109.0, "place_name":"Barrio Norte","place_with_parent_names": "	|Argentina|Capital Federal|Barrio Norte|","lat-lon" : "-34.593104, -58.400381", "lat": -34.593104, "lon": -58.400381}
datos_barrio_norte_completo = datos_barrio_norte.fillna(values_3436109)
#datos_barrio_norte_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_barrio_norte_completo)
geo_nulos.isnull().sum()

### LIMPIEZA RAMOS MEJIA

In [ ]:
mask_ramos_mejia = geo_nulos['place_name']== "Ramos Mejía"
datos_ramos_mejia = geo_nulos.loc[mask_ramos_mejia, :]
#datos_ramos_mejia.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_ramos_mejia.isnull().sum()

In [ ]:
#display(datos_ramos_mejia.groupby(['geonames_id']).mean())

In [ ]:
values_3429617 = {"geonames_id":3429617.0, "place_name":"Ramos Mejía","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...","lat-lon" : "-34.648363, -58.562762", "lat": -34.648363, "lon": -58.562762}
datos_ramos_mejia_completo = datos_ramos_mejia.fillna(values_3429617)
#datos_barrio_norte_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_ramos_mejia_completo)
geo_nulos.isnull().sum()

### LIMPIEZA MARTINEZ

In [ ]:
mask_martinez = geo_nulos['place_name']== "Martínez"
datos_martinez = geo_nulos.loc[mask_martinez, :]
#datos_martinez.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_martinez.isnull().sum()

In [ ]:
#display(datos_martinez.groupby(['geonames_id']).mean())

In [ ]:
values_3430813 = {"geonames_id":3430813.0, "place_name":"Martínez","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...","lat-lon" : "-34.492594, -58.506898", "lat": 	-34.492594, "lon": -58.506898}
datos_martinez_completo = datos_martinez.fillna(values_3430813)
#datos_martinez_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_martinez_completo)
geo_nulos.isnull().sum()

### LIMPIEZA LANUS

In [ ]:
mask_lanus = geo_nulos['place_name']== "Lanús"
datos_lanus = geo_nulos.loc[mask_lanus, :]
#datos_lanus.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_lanus.isnull().sum()

In [ ]:
#display(datos_lanus.groupby(['geonames_id']).mean())

In [ ]:
values_3432134 = {"geonames_id":3432134.0, "place_name":"Lanús","place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Sur|Lanús|Lanús|","lat-lon" : "-34.701389, -58.398743", "lat": -34.701389, "lon": -58.398743}
datos_lanus_completo = datos_lanus.fillna(values_3432134)
#datos_lanus_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_lanus_completo)
geo_nulos.isnull().sum()

### LIMPIEZA ESCOBAR

In [ ]:
mask_escobar = geo_nulos['place_name']== "Escobar"
datos_escobar = geo_nulos.loc[mask_escobar, :]
#datos_escobar.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_escobar.isnull().sum()

In [ ]:
display(datos_escobar.groupby(['geonames_id']).mean())

In [ ]:
values_3434130 = {"geonames_id":3434130.0, "place_name":"Escobar", "place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|","lat-lon" : "-34.36929, -58.756345", "lat": -34.36929, "lon": -58.756345}
datos_escobar_completo = datos_escobar.fillna(values_3434130)
#datos_escobar_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_escobar_completo)
geo_nulos.isnull().sum()

### LIMPIEZA BANFIELD

In [ ]:
mask_banfield = geo_nulos['place_name']== "Banfield"
datos_banfield = geo_nulos.loc[mask_banfield, :]
#datos_banfield.sort_values(by="geonames_id").sample(10)

In [ ]:
datos_banfield.isnull().sum()

In [ ]:
#display(datos_banfield.groupby(['geonames_id']).mean())

In [ ]:
values_3436152 = {"geonames_id":3436152.0, "place_name":"Banfield", "place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...","lat-lon" : "-34.742923, -58.399492", "lat": -34.742923, "lon": -58.399492}
datos_banfield_completo = datos_banfield.fillna(values_3436152)#
#datos_banfield_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_banfield_completo)
geo_nulos.isnull().sum()

### LIMPIEZA SAN ISIDRO

In [ ]:
mask_sa_isidro = geo_nulos['place_name']== "San Isidro"
datos_san_isidro = geo_nulos.loc[mask_sa_isidro, :]
#datos_san_isidro.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_san_isidro.isnull().sum()

In [ ]:
#display(datos_san_isidro.groupby(['geonames_id']).mean())

In [ ]:
values_34289983 = {"geonames_id":3428983.0, "place_name":"San Isidro", "place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|","lat-lon" : "-34.478067, -58.536362", "lat": -34.478067, "lon": -58.536362}
datos_san_isidro_completo = datos_san_isidro.fillna(values_34289983)
#datos_san_isidro_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_san_isidro_completo)
geo_nulos.isnull().sum()

### LIMPIEZA PINAMAR

In [ ]:
mask_pinarmar = geo_nulos['place_name']== "Pinamar"
datos_pinamar = geo_nulos.loc[mask_pinarmar, :]
datos_pinamar.sort_values(by="geonames_id").sample(10)

In [ ]:
datos_pinamar.isnull().sum()

In [ ]:
display(datos_pinamar.groupby(['geonames_id']).mean())

In [ ]:
values_3429971 = {"geonames_id":3429971.0, "place_name":"Pinamar", "place_with_parent_names": "|Argentina|Buenos Aires Costa Atlántica|Pinamar|","lat-lon" : "-37.10956	-56.860607", "lat": -37.10956, "lon": -56.860607}
datos_pinamar_completo = datos_pinamar.fillna(values_3429971)
datos_pinamar_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_pinamar_completo)
geo_nulos.isnull().sum()

### LIMPIEZA BOEDO

In [ ]:
mask_boedo = geo_nulos['place_name']== "Boedo"
datos_boedo = geo_nulos.loc[mask_boedo, :]
#datos_boedo.sort_values(by="geonames_id").sample(10)

In [ ]:
datos_boedo.isnull().sum()

In [ ]:
display(datos_boedo.groupby(['geonames_id']).mean())

In [ ]:
values_3436003 = {"geonames_id":3436003.0, "place_name":"Boedo", "place_with_parent_names": "|Argentina|Capital Federal|Boedo|","lat-lon" : "-34.625496, -58.417307", "lat": -34.625496, "lon": -58.417307}
datos_boedo_completo = datos_boedo.fillna(values_3436003)
datos_boedo_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_boedo_completo)
geo_nulos.isnull().sum()

### LIMPIEZA NUÑEZ

In [ ]:
mask_nunez = geo_nulos['place_name']== "Nuñez"
datos_nunez = geo_nulos.loc[mask_nunez, :]
#datos_nunez.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_nunez.isnull().sum()

In [ ]:
#display(datos_nunez.groupby(['geonames_id']).mean())

In [ ]:
values_3430348 = {"geonames_id":3430348.0, "place_name":"Nuñez", "place_with_parent_names": "|Argentina|Capital Federal|Nuñez|","lat-lon" : "-34.549629, -58.463268", "lat": -34.549629, "lon": -58.463268}
datos_nunez_completo = datos_nunez.fillna(values_3430348)
#datos_nunez_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_nunez_completo)
geo_nulos.isnull().sum()

### LIMPIEZA SAN MIGUEL

In [ ]:
mask_sa_miguel = geo_nulos['place_name']== "San Miguel"
datos_san_miguel = geo_nulos.loc[mask_sa_miguel, :]#
#datos_san_miguel.sort_values(by="geonames_id").sample(10)

In [ ]:
#datos_san_miguel.isnull().sum()

In [ ]:
display(datos_san_miguel.mean())

In [ ]:
geoname = 34645366.0

In [ ]:
values_34645366 = {"geonames_id":34645366.0, "place_name":"San Miguel", "place_with_parent_names": "|Argentina|Bs.As. G.B.A. Zona Norte|San Miguel|","lat-lon" : " -34.542439, -58.713420", "lat":  -34.542439, "lon":  -58.713420}
datos_san_miguel_completo = datos_san_miguel.fillna(values_3430348)
#datos_san_miguel_completo.head()

In [ ]:
geo_nulos = geo_nulos.fillna(datos_san_miguel_completo)
geo_nulos.isnull().sum()

In [ ]:
data[['geonames_id_nuevo', 'lat-lon_nuevo', 'lat_nuevo', 'lon_nuevo']] = geo_nulos[['geonames_id', 'lat-lon', 'lat', 'lon']]

<a id="section_limpieza_pisos"></a> 

### Limpieza de la columna Pisos

[volver a Indice](#section_indice)


In [ ]:
#Creación de una nueva columna con la copia de los valores de la columna 'floor'
data['floor_copia'] = data['floor']
data['property_type_copia'] = data['property_type']
#Cambio a minúscula estas dos nuevas columnas creadas
data['description_copia'] = data['description'].astype(str).str.lower()
data['title_copia']= data['title'].astype(str).str.lower()

In [ ]:
#Comprobación de valores null en data 'floor_copia'
data['floor_copia'].isnull().value_counts()

In [ ]:
#Comprobación de valores null en data 'description_copia'
data['description_copia'].isnull().value_counts()

In [ ]:
#Comprobación de valores null en data 'title_copia'
data['title_copia'].isnull().value_counts()

In [ ]:
#Categrias en property_type y qué cantidaa hay
data['property_type_copia'].value_counts()

In [ ]:
#agrupo por tipo de propiedad
tipo_propiedad = data.groupby('property_type_copia')

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)

In [ ]:
pisos.value_counts()

In [ ]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
patron = "\spiso:[\s]?(?P<numero>\d[\d]?)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
patron = "piso[\s]?(?P<numero>\d[\d]?)"

busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "\s(?P<numero>\d[\d]?)[\s]?[º|°]?\s*piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data.floor_copia.dtypes

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "\s(?P<numero>\d[\d]?)\s*\w{0,5}\s*piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data.floor_copia.dtypes

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(duplex|triplex)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'duplex': 2,'triplex': 3}
encontradas1 = encontradas.map(numcol_cat_mapper)


In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(primer|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)[\s]?piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1,'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"[\s]?piso[\s]?(primer[a|o]?|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1, 'primera': 1, 'primero': 1, 'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?piso[\s]"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?[sola]?[\s]?planta[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 0,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"planta[\s]?(un[a|o]|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 1, 'uno': 1, 'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(planta[\s]?alta)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta alta': 1,'plantalata': 1}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(planta[\s]?baja)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta baja': 0,'plantabaja': 0}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
mascara = data['description_copia'] == ""

Ahora busco con las mismas regex pero con la columna 'title_copia'

In [ ]:
patron = r"(planta[\s]?baja)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta baja': 0,'plantabaja': 0}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?[sola]?[\s]?planta[s]?"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 0,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?piso[\s]"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
patron = r"(primer|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)[\s]?piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1,'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(duplex|triplex)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'duplex': 2,'triplex': 3}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "\s(?P<numero>\d[\d]?)\s*\w{0,5}\s*piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "\s(?P<numero>\d[\d]?)[\s]?[º|°]?\s*piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "piso[\s]?(?P<numero>\d[\d]?)"

busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = "\spiso:[\s]?(?P<numero>\d[\d]?)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[data.title_copia!= ""]

In [ ]:
data.loc[121217, 'description_copia']

In [ ]:
patron = r"(casa[s]?|chalet|quinta[s]?|cabaña[s]?|pb)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'casa': 0,'casas':0, 'chalet': 0, 'quinta': 0, 'quintas': 0, 'cabaña': 0, \
                    'cabañas':0, 'pb': 0 }
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
patron = r"(casa[s]?|chalet|quinta[s]?|cabaña[s]?|pb)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'casa': 0,'casas':0, 'chalet': 0, 'quinta': 0, 'quintas': 0, 'cabaña': 0, \
                    'cabañas':0, 'pb': 0 }
encontradas1 = encontradas.map(num_pisos_mapper)

In [ ]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [ ]:
data['floor_copia'].isnull().value_counts()

In [ ]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [ ]:
data.sample(3)

<a id="section_limpieza_antiguedad"></a> 

### Limpieza de la columna Antigüedad

[volver a Indice](#section_indice)


In [ ]:
data_mask = data.description.notna()
data_con_descriptions =  data.loc[data_mask,:]
data_con_descriptions.shape

In [ ]:
#Análisis de inmuebles a estrenar
a_estrenar = data_con_descriptions[data_con_descriptions['description'].str.contains('estrenar')]
mask_a_estrenar = data.index.isin(a_estrenar.index)

In [ ]:
#Análisis de años de antiguedad en número
pattern_antiguedad = '(\d+)[\s]año'
pattern_antiguedad_regex = re.compile(pattern_antiguedad)
description_series = data_con_descriptions.description
resultado_antiguedad = description_series.apply(lambda x: pattern_antiguedad_regex.search(x))
resultado_antiguedad[resultado_antiguedad.notnull()]
encontradas_ant = resultado_antiguedad[resultado_antiguedad.notnull()].apply(lambda x: x.group(1))

In [ ]:
encontradas_ant.shape

#### Creamos variable ant para identificar si es a estrenar, de pozo u otros

In [ ]:
data.loc[mask_a_estrenar, 'ant'] = 'a_estrenar'
data.loc[data['ant'].isnull(),'ant'] = 'otros'
data.ant.value_counts()

In [ ]:
print(a_estrenar.state_name.value_counts(normalize = True))
data.state_name.value_counts(normalize = True)

In [ ]:
data.price.describe().apply(lambda x: format(x, 'f'))
a_estrenar.price.describe().apply(lambda x: format(x, 'f'))

<a id="section_limpieza_precios"></a> 

### Limpieza de la columna Precios

[volver a Indice](#section_indice)


In [ ]:
data_precio= data.loc[:,["place_name","price", "currency", "price_aprox_local_currency", "price_aprox_usd","surface_total_in_m2", "surface_covered_in_m2","price_usd_per_m2", "price_per_m2", "property_type"]]
data_precio

In [ ]:
data_precio.isnull().sum()

In [ ]:
tc = data_precio["price_aprox_local_currency"] / data_precio["price"]
tc[:1]

## Datos basicos para el analisis:

1 - Columna Currency cambiar ARS X USD


In [ ]:
mask_ars = data_precio['currency']== "ARS"
datos_ars =data_precio.loc[mask_ars, :]
datos_ars

In [ ]:
relacion  = datos_ars["price_aprox_local_currency"]/tc
relacion[mask_ars]

In [ ]:
data_precio_1 = data_precio.replace("ARS", value = "USD")

In [ ]:
data_precio_1.isnull().sum()

2 - Ajusto columna local price de acuerdo a TC

In [ ]:
data_precio_1["price_aprox_local_currency_nuevo"] = data_precio_1["price_aprox_local_currency"] = data_precio_1["price"]*tc
data_precio_1.replace("price_aprox_local_currency","price_aprox_local_currency_nuevo" )

In [ ]:
data_precio_1.drop(labels="price_aprox_local_currency_nuevo", axis=1)

In [ ]:
data_precio_1["surface_total_in_m2"].replace("Nan",data_precio_1["surface_covered_in_m2"] )

In [ ]:
data_precio_1["price_usd_per_m2"] = data_precio_1["price"]/data_precio_1["surface_total_in_m2"]
data_precio_1["price_per_m2"] = data_precio_1["price_aprox_local_currency"]/data_precio_1["surface_total_in_m2"]
data_precio_1

In [ ]:
data_precio_1.isnull().sum()

In [ ]:
mask=data_precio_1["price"].notnull()
con_precios = data_precio_1.loc[mask, :]
con_precios

In [ ]:
data[['price_nuevo', 'currency_nuevo', 'price_aprox_local_currency_nuevo', 'price_aprox_usd_nuevo', 'surface_total_in_m2_nuevo',\
      'surface_covered_in_m2_nuevo', 'price_usd_per_m2_nuevo', 'price_per_m2_nuevo']] = data_precio_1.loc[:,'price':'price_per_m2']

In [ ]:
data.columns

<a id="section_limpieza_places"></a> 

### Limpieza de la columna Places

[volver a Indice](#section_indice)


In [ ]:
estructura_places = pd.DataFrame(data.place_with_parent_names.str.split('|', 6).tolist(),
                                   columns = ['1', 'pais','provincia','localidad','zona','barrio','lugar'])

#### Tomamos los datos "None" y los transformamos en vacíos

In [ ]:
estructura_places.fillna(value="", inplace=True)

#### Tomamos los datos vacíos y copiamos la información del "padre" directo

In [ ]:
estructura_places.loc[estructura_places["localidad"] == '','localidad'] = estructura_places["provincia"]
estructura_places.loc[estructura_places["zona"] == '','zona'] = estructura_places["localidad"]
estructura_places.loc[estructura_places["barrio"] == '','barrio'] = estructura_places["zona"]
estructura_places.loc[estructura_places["lugar"] == '','lugar'] = estructura_places["barrio"]
estructura_places.head(3)

#### Joineamos la el dataset con el original

In [ ]:
data = data.join(estructura_places)

### Analizamos el resultado final y eliminamos las variables duplicadas e inncesarias

In [ ]:
data.dtypes

In [ ]:
data.drop(['Unnamed: 0', 'operation', 'property_type_copia', 'place_with_parent_names', 'country_name', 'geonames_id', 'lat-lon',\
          'lat', 'lon','1', 'price', 'currency', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',\
          'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor', 'rooms', 'expenses',\
          'properati_url', 'image_thumbnail'], axis =1, inplace = True)

In [ ]:
pd.options.display.max_columns = None
data.head(3)

<a id="section_mitos_o_realidades"></a> 

## Definición de Mitos o Realidades

[volver a Indice](#section_indice)

Nos planteamos algunas preguntas en forma de mito o realidad con el objetivo de establecer a partir de los datos si existe la posibilidad de afirmar o refutar el mito.
Para todos los análisis utilizamos información solo del AMBA, identificado a través de la variable 'state_name'.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

<a id="section_mito_1"></a> 

### Mito o Realidad N° 1: Los inmuebles a estrenar son más caros que los demás

[volver a Indice](#section_indice)


In [ ]:
#Nos quedamos solo con la info del AMBA
data_amba = data[data['state_name'].isin(['Bs.As. G.B.A. Zona Norte', 'Bs.As. G.B.A. Zona Oeste', 'Capital Federal',\
                                         'Bs.As. G.B.A. Zona Sur'])]

In [ ]:
#Elimino del análisis los outliers de precios

q1 = data_amba.price_nuevo.quantile(0.25)
print(q1)
q2 = data_amba.price_nuevo.quantile(0.5)
print(q2)
q3 = data_amba.price_nuevo.quantile(0.75)
print(q3)


iqr = (q3 - q1) * 1.5

limite_superior_precio = q3 + iqr
limite_inferior_precio = q1 - iqr

print(limite_superior_precio)
print(limite_inferior_precio)


outlier_mask_sup_precio = data_amba.price_nuevo > limite_superior_precio
outlier_mask_inf_precio = data_amba.price_nuevo < limite_inferior_precio
outlier_mask_precio = np.logical_or(outlier_mask_sup_precio, outlier_mask_inf_precio)
not_outliers_precio = np.logical_not(outlier_mask_precio)
data_amba_sin_outliers_precio = data_amba[not_outliers_precio]

#Eliminamos del análisis los tipo de propiedad Store
data_amba_sin_outliers_sin_negocios_precio = data_amba_sin_outliers_precio[(data_amba_sin_outliers_precio.property_type != 'store')]
#Eliminamos del análisis los casos que tienen más de 5 habitaciones (el limite superior para identificar outliers es de 3.5 habitaciones).
data_amba_sin_outliers_sin_negocios_menos_cinco_hab_precio = data_amba_sin_outliers_sin_negocios_precio[(data_amba_sin_outliers_sin_negocios_precio.rooms_completo < 6)]

In [ ]:
#Graficamos el precio promedio de las viviendas a estrenar vs las que no lo son por la cantidad de habitaciones
ax = sns.barplot(x="rooms_completo", y="price_nuevo", hue="ant", data=data_amba_sin_outliers_sin_negocios_menos_cinco_hab_precio)

In [ ]:
g = sns.FacetGrid(data_amba_sin_outliers_sin_negocios_menos_cinco_hab_precio, col = "property_type", height=7, aspect=.6)
g.map(sns.barplot, "ant", "price_nuevo")

In [ ]:
g = sns.FacetGrid(data_amba_sin_outliers_sin_negocios_menos_cinco_hab_precio, col = "ant", height=7, aspect=.9)
g.map(sns.barplot, "rooms_completo", "price_nuevo");

#### Conclusion: No se puede establecer que los inmuebles a estrenar tienen un valor significativamente mayor con los datos obtenidos.

<a id="section_mito_2"></a> 

### Mito o Realidad N° 2: Existe una correlación entre el precio del inmueble y las expensas que abonan

[volver a Indice](#section_indice)


In [ ]:
#Elimino del análisis los outliers de expensas

q1 = data_amba_sin_outliers_precio.expenses_nuevo.quantile(0.25)
print(q1)
q2 = data_amba_sin_outliers_precio.expenses_nuevo.quantile(0.5)
print(q2)
q3 = data_amba_sin_outliers_precio.expenses_nuevo.quantile(0.75)
print(q3)


iqr = (q3 - q1) * 1.5

limite_superior_expensas = q3 + iqr
limite_inferior_expensas = q1 - iqr

print(limite_superior_expensas)
print(limite_inferior_expensas)


outlier_mask_sup_expensas = data_amba_sin_outliers_precio.expenses_nuevo > limite_superior_expensas
outlier_mask_inf_expensas = data_amba_sin_outliers_precio.expenses_nuevo < limite_inferior_expensas
outlier_mask_expensas = np.logical_or(outlier_mask_sup_expensas, outlier_mask_inf_expensas)
not_outliers_expensas = np.logical_not(outlier_mask_expensas)
data_amba_sin_outliers_expensas = data_amba_sin_outliers_precio[not_outliers_expensas]

In [ ]:
sns.pairplot(data_amba_sin_outliers_expensas.loc[:, ('expenses_nuevo', 'price_nuevo', 'state_name', 'property_type')], hue="property_type")

#### Conclusion: Gráficamente no se observa relación entre el precio de la propiedad y las expensas que abona.

In [ ]:
# with sns.axes_style(style='ticks'):
#     g = sns.catplot("property_type", "price_nuevo", "state_name", data=data_amba_sin_outliers_precio, kind="box")
#     g.set_axis_labels("Tipo de vivienda", "Precio de vivienda");